# **Final Project**

This is a project doing some basic data analysis of IMDB movie data and associated wiki streaming events. It should be completed by groups of no less than 2 students and no more than 4 students. Each member of the group should have at least a few commits associated in the project repo.

## **Scoring**

The code must run and provide the correct answers . 1/2 points
The remainder will come from notebook organization, code comments, etc .
For the questions that have answers, please also provide those in markdown cells in the notebook, and/or part of a mardown file in the repo .
All relevant code should be shared via a shared Git repository. Additionally, you will send an email to joe@adaltas.com when the project has been submitted . Please ensure that the names of all participants are included in the repo and in the submission email . Note: For full credit the code must run with little to no extra input from the end user, and, any extra input that is required must be clearly documented and explained. Also note, any question that is at least attempted will be awarded with partial credit provided there is a corresponding explanation of the difficulties faced.

## **Questions**

  1 - load data from here. This should be done using a notebook cell and not a manual process to import the data. NOTE: You may not need all of the datasets, but you will be utilizing most of them.

  2 - How many total people in data set?

  3 - What is the earliest year of birth?

  4 - How many years ago was this person born?

  5 - Using only the data in the data set, determine if this date of birth correct.

  6 - Explain the reasoning for the answer in a code comment or new markdown cell.

  7 - What is the most recent data of birth?

  8 - What percentage of the people do not have a listed date of birth?

  9 - What is the length of the longest "short" after 1900?

  10 - What is the length of the shortest "movie" after 1900?

  11 - List of all of the genres represented.

  12 - What is the higest rated comedy "movie" in the dataset? Note, if there is a tie, the tie shall be broken by the movie with the most votes.

  13 - Who was the director of the movie?

  14 - List, if any, the alternate titles for the movie.

## **Stream Processing**

Choose any five entities from the data set. These can be specific movies, actors, crews, etc, or more abstract concepts such as specific genres, etc. The main criteria is that the entities chosen must have a trackable wiki page. Set up a stream processing job that will track events for the chosen entities from the wikimedia Events Platform. These tracking jobs should provide some simple metrics. These metrics should be stored in a database or file (depending on the platform used). At least one of the metrics should be of the "alert" type (meaning some event that would require further action. For instance imagine wanting to be notified each time a specific user makes a change. Capture this "alert" and mimic an alerting system by routing these events to a different file/database.) These tables/data do not need to be shared, but the structure of the output should be clearly noted in the code and/or markdown cells. Additionally, a brief explanation/overview of this section should be provided in a seperate markdown cell or in the project readme.

---

## **Population Script**

In [1]:
import os

local_path = "/tmp/big-data-processing-project/data"

os.makedirs(local_path, exist_ok=True)
local_path

# 2. Download files using shell 
files = [
    "name.basics.tsv.gz",
    "title.akas.tsv.gz",
    "title.basics.tsv.gz",
    "title.crew.tsv.gz",
    "title.episode.tsv.gz",
    # "title.principals.tsv.gz",                        # Too big for instance to run it in Databricks
    "title.ratings.tsv.gz"
]

dict_files_names = {
    "name.basics.tsv.gz": "name.basics",
    "title.akas.tsv.gz": "title.akas",
    "title.basics.tsv.gz": "title.basics",
    "title.crew.tsv.gz": "title.crew",
    "title.episode.tsv.gz": "title.episode",
    # "title.principals.tsv.gz": "title.principals",    # Too big for instance to run it in Databricks
    "title.ratings.tsv.gz": "title.ratings"
}

base_url = "https://datasets.imdbws.com/"

for f in files:
    url = base_url + f
    out = f"{local_path}/{f}"
    print("Downloading:", f)
    os.system(f"wget -O {out} {url}")

Downloading: name.basics.tsv.gz


--2025-12-08 13:48:50--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 2600:9000:203b:fa00:3:3082:af00:93a1, 2600:9000:203b:8a00:3:3082:af00:93a1, 2600:9000:203b:d400:3:3082:af00:93a1, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|2600:9000:203b:fa00:3:3082:af00:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 295454483 (282M) [binary/octet-stream]
Saving to: ‘/tmp/big-data-processing-project/data/name.basics.tsv.gz’

     0K .......... .......... .......... .......... ..........  0% 6.11M 46s
    50K .......... .......... .......... .......... ..........  0% 11.7M 35s
   100K .......... .......... .......... .......... ..........  0% 16.8M 29s
   150K .......... .......... .......... .......... ..........  0% 10.2M 29s
   200K .......... .......... .......... .......... ..........  0% 37.0M 24s
   250K .......... .......... .......... .......... ..........  0% 31.4M 22s
   300K ........

Downloading: title.akas.tsv.gz


connected.
HTTP request sent, awaiting response... 200 OK
Length: 471672368 (450M) [binary/octet-stream]
Saving to: ‘/tmp/big-data-processing-project/data/title.akas.tsv.gz’

     0K .......... .......... .......... .......... ..........  0% 6.93M 65s
    50K .......... .......... .......... .......... ..........  0% 18.5M 45s
   100K .......... .......... .......... .......... ..........  0% 9.01M 46s
   150K .......... .......... .......... .......... ..........  0% 18.2M 41s
   200K .......... .......... .......... .......... ..........  0% 38.7M 35s
   250K .......... .......... .......... .......... ..........  0% 22.2M 33s
   300K .......... .......... .......... .......... ..........  0% 28.0M 30s
   350K .......... .......... .......... .......... ..........  0% 25.5M 29s
   400K .......... .......... .......... .......... ..........  0% 30.8M 27s
   450K .......... .......... .......... .......... ..........  0% 68.0M 25s
   500K .......... .......... .......... .......... ...

Downloading: title.basics.tsv.gz


...

.... .......... .......... ..........  0% 7.43M 10s
    50K .......... .......... .......... .......... ..........  0% 14.3M 8s
   100K .......... .......... .......... .......... ..........  0% 9.63M 8s
   150K .......... .......... .......... .......... ..........  0% 10.6M 8s
   200K .......... .......... .......... .......... ..........  0% 37.0M 6s
   250K .......... .......... .......... .......... ..........  0%  456M 5s
   300K .......... .......... .......... .......... ..........  0% 26.8M 5s
   350K .......... .......... .......... .......... ..........  0%  678M 4s
   400K .......... .......... .......... .......... ..........  0% 11.6M 5s
   450K .......... .......... .......... .......... ..........  0% 48.6M 4s
   500K .......... .......... .......... .......... ..........  0% 54.3M 4s
   550K .......... .......... .......... .......... ..........  0%  939M 4s
   600K .......... .......... .......... .......... ..........  0% 48.3M 4s
   650K .......... .......... ......

Downloading: title.episode.tsv.gz


.... 10% 78.0M 1s
  5350K .......... .......... .......... .......... .......... 10%  729M 1s
  5400K .......... .......... .......... .......... .......... 10%  247M 1s
  5450K .......... .......... .......... .......... .......... 10% 85.7M 1s
  5500K .......... .......... .......... .......... .......... 10% 68.1M 1s
  5550K .......... .......... .......... .......... .......... 11% 11.4M 1s
  5600K .......... .......... .......... .......... .......... 11% 21.4M 1s
  5650K .......... .......... .......... .......... .......... 11% 96.1M 1s
  5700K .......... .......... .......... .......... .......... 11%  187M 1s
  5750K .......... .......... .......... .......... .......... 11% 29.5M 1s
  5800K .......... .......... .......... .......... .......... 11%  581M 1s
  5850K .......... .......... .......... .......... .......... 11%  108M 1s
  5900K .......... .......... .......... .......... .......... 11% 63.0M 1s
  5950K .......... .......... .......... .......... .......... 11%  72

Downloading: title.ratings.tsv.gz


......... ..........  0% 6.91M 1s
    50K .......... .......... .......... .......... ..........  1% 10.3M 1s
   100K .......... .......... .......... .......... ..........  1% 12.2M 1s
   150K .......... .......... .......... .......... ..........  2% 12.3M 1s
   200K .......... .......... .......... .......... ..........  3% 35.4M 1s
   250K .......... .......... .......... .......... ..........  3% 73.0M 1s
   300K .......... .......... .......... .......... ..........  4% 19.0M 1s
   350K .......... .......... .......... .......... ..........  5% 30.6M 0s
   400K .......... .......... .......... .......... ..........  5% 40.6M 0s
   450K .......... .......... .......... .......... ..........  6% 35.4M 0s
   500K .......... .......... .......... .......... ..........  6% 84.5M 0s
   550K .......... .......... .......... .......... ..........  7%  349M 0s
   600K .......... .......... .......... .......... ..........  8% 21.3M 0s
   650K .......... .......... .......... .......... ..

In [6]:
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home"
print("JAVA_HOME =", os.environ.get("JAVA_HOME"))
os.system("java -version")
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("IMDB Analysis").getOrCreate()
dict_df = {}
for file_name in files:
    df = (spark.read
          .option("compression", "gzip")
          .option("inferSchema", "false")
          .option("nullValue", "\\N")
          .csv(f"{local_path}/{file_name}", header=True, sep="\t"))
    df.show(5)
    dict_df[dict_files_names[file_name]] = df

JAVA_HOME = /opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home
+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|actor,miscellaneo...|tt0072308,tt00504...|
|nm0000002|  Lauren Bacall|     1924|     2014|actress,miscellan...|tt0037382,tt00752...|
|nm0000003|Brigitte Bardot|     1934|     NULL|actress,music_dep...|tt0057345,tt00491...|
|nm0000004|   John Belushi|     1949|     1982|actor,writer,musi...|tt0072562,tt00779...|
|nm0000005| Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050986,tt00694...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 5 rows


openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment Homebrew (build 17.0.17+0)
OpenJDK 64-Bit Server VM Homebrew (build 17.0.17+0, mixed mode, sharing)


+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Carmencita|  NULL|    NULL|   original|         NULL|              1|
|tt0000001|       2|          Carmencita|    DE|    NULL|       NULL|literal title|              0|
|tt0000001|       3|          Carmencita|    US|    NULL|imdbDisplay|         NULL|              0|
|tt0000001|       4|Carmencita - span...|    HU|    NULL|imdbDisplay|         NULL|              0|
|tt0000001|       5|          Καρμενσίτα|    GR|    NULL|imdbDisplay|         NULL|              0|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
only showing top 5 rows
+---------+---------+--------------------+--------------------+-------+-----

---

## **Questions**

**2.**

In [7]:
unique_name_count = (
  dict_df["name.basics"]
  .select("primaryName")
  .distinct()
  .count()
)
display(unique_name_count)

11397879

**3.**

There is a major problem here since the original dataset provides us with dates in absolute values as the example below proves it with the date of birth of Cesar (-100 -> 100).

In [8]:
raw = spark.read.text(f"{local_path}/name.basics.tsv.gz")
raw.filter(raw.value.contains("Gaio Giulio Cesare")).show(20, False)

+-----------------------------------------------------------------------------------+
|value                                                                              |
+-----------------------------------------------------------------------------------+
|nm2471712\tGaio Giulio Cesare\t100\t44\twriter,archive_footage\ttt0191909,tt0057105|
+-----------------------------------------------------------------------------------+



-> We have then the date of birth closest to 0 in the code below.

In [9]:
from pyspark.sql.functions import col, min

min_birth_year = (
  dict_df["name.basics"]
  .select(min(col("birthYear").cast("double")))
  .collect()[0][0]
)

min_birth_year_df = dict_df["name.basics"].filter(col("birthYear") == min_birth_year)

min_birth_year_df.show(5)

+---------+------------------+---------+---------+-----------------+--------------------+
|   nconst|       primaryName|birthYear|deathYear|primaryProfession|      knownForTitles|
+---------+------------------+---------+---------+-----------------+--------------------+
|nm0784172|Lucio Anneo Seneca|        4|       65|           writer|tt0043802,tt02188...|
+---------+------------------+---------+---------+-----------------+--------------------+



**4.**

In [10]:
from datetime import date

current_year = date.today().year
years_difference = int(current_year - min_birth_year)
print(f"The difference between the current year and the earliest date of birth in our dataset is {years_difference} years!")

The difference between the current year and the earliest date of birth in our dataset is 2021 years!


**5.**

In [0]:
from pyspark.sql.functions import col

person_nconst = min_birth_year_df.select("nconst").first()[0]

person_works = (
    dict_df["title.crew"]
    .filter(col("directors").contains(person_nconst) | col("writers").contains(person_nconst))
    .join(dict_df["title.basics"], on="tconst")
    .select("startYear")
    .filter(col("startYear").isNotNull())
    .orderBy("startYear")
)

first_work = person_works.first()
if first_work:
    age = int(first_work[0]) - int(min_birth_year)
    verdict = "INCORRECT" if age < 0 else "SUSPICIOUS" if age < 10 or age > 150 else "plausible"
    print(f"Age at first work: {age} years - Birth year seems {verdict}")
else:
    print("No works found to verify birth year")

**6.**

- If we consider this question as a question about the veracity of the earliest date of birth in this dataset, we have answered it a bit above (indicating that the date values were absolute -> preventing us from finding the earliest one but allowing us to find the closest to 0)

- On another hand, if we consider this question as a question about how we can check the veracity of the date of birth of this person

**7.**

In [11]:
from pyspark.sql.functions import col, max

max_birth_year = (
  dict_df["name.basics"]
  .select(max(col("birthYear").cast("double")))
  .collect()[0][0]
)

max_birth_year_df = dict_df["name.basics"].filter(col("birthYear") == max_birth_year)

max_birth_year_df.show(5)

+----------+-----------------+---------+---------+--------------------+--------------------+
|    nconst|      primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+----------+-----------------+---------+---------+--------------------+--------------------+
|nm16784939|Kyrah Ivy Jackson|     2025|     NULL|             actress|                NULL|
| nm5642311|     Chase Ramsey|     2025|     NULL|actor,director,wr...|tt17505010,tt1471...|
+----------+-----------------+---------+---------+--------------------+--------------------+



**8.**

In [12]:
from pyspark.sql.functions import col

total_rows = dict_df["name.basics"].count()
null_rows = dict_df["name.basics"].filter(col("birthYear").isNull()).count()

birth_year_null_pct = (null_rows / total_rows) * 100

print(f"{birth_year_null_pct:.2f}% of the people in this dataset do not have a listed date of birth!")

95.57% of the people in this dataset do not have a listed date of birth!


**9.**

In [13]:
from pyspark.sql.functions import col, max

longest_short_after_1900 = (
  dict_df["title.basics"]
  .filter((col("titleType") == "short") & (col("startYear") >= 1900))
  .select(max(col("runtimeMinutes")))
  .collect()[0][0]
)

print(f"The longest short film after 1900 was {longest_short_after_1900} minutes long!")

The longest short film after 1900 was 97 minutes long!


**10.**

In [14]:
from pyspark.sql.functions import col, min

shortest_movie_after_1900 = (
  dict_df["title.basics"]
  .filter((col("titleType") == "movie") & (col("startYear") >= 1900))
  .select(min(col("runtimeMinutes")))
  .collect()[0][0]
)

print(f"The shortest movie film after 1900 was {shortest_movie_after_1900} minutes long!")

The shortest movie film after 1900 was 1 minutes long!


**11.**

In [15]:
from pyspark.sql.functions import split, explode, trim, col

genres_df = (
    dict_df["title.basics"]
    .select(explode(split(col("genres"), ",")).alias("genre"))
    .select(trim(col("genre")).alias("genre"))
    .filter(col("genre").isNotNull() & (col("genre") != ""))
)

unique_genres = [row["genre"] for row in genres_df.select("genre").distinct().collect()]

print(unique_genres)

['Crime', 'Romance', 'Thriller', 'Adventure', 'Drama', 'War', 'Documentary', 'Reality-TV', 'Family', 'Fantasy', 'Game-Show', 'Adult', 'History', 'Mystery', 'Musical', 'Animation', 'Music', 'Film-Noir', 'Short', 'Horror', 'Western', 'Biography', 'Comedy', 'Sport', 'Action', 'Talk-Show', 'Sci-Fi', 'News']


**12.**

In [16]:
from pyspark.sql.functions import col, desc, dense_rank
from pyspark.sql.window import Window

df_joined = dict_df["title.basics"].join(
    dict_df["title.ratings"],
    on="tconst",
    how="inner"
)

df_filtered = df_joined.filter((col("titleType") == "movie") & (col("genres").contains("Comedy")))
w = Window.orderBy(desc("averageRating"), desc("numVotes"))

highest_rated_comedy_movie = (
    df_filtered
    .withColumn("rank", dense_rank().over(w))
    .filter(col("rank") == 1)
    .drop("rank")
)

highest_rated_comedy_movie.show(truncate=False)

25/12/08 13:52:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/08 13:52:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/08 13:52:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/08 13:52:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/08 13:52:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/08 13:52:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/08 1

+----------+---------+------------+-------------+-------+---------+-------+--------------+--------------+-------------+--------+
|tconst    |titleType|primaryTitle|originalTitle|isAdult|startYear|endYear|runtimeMinutes|genres        |averageRating|numVotes|
+----------+---------+------------+-------------+-------+---------+-------+--------------+--------------+-------------+--------+
|tt25967770|movie    |Zucchini    |Zucchini     |0      |2025     |NULL   |83            |Comedy,Romance|9.9          |9       |
+----------+---------+------------+-------------+-------+---------+-------+--------------+--------------+-------------+--------+



25/12/08 13:52:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/08 13:52:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


**13.**

In [17]:
highest_rated_comedy_movie_director_df = (
    highest_rated_comedy_movie
    .join(dict_df["title.crew"], on="tconst", how="inner")
    .join(dict_df["name.basics"], dict_df["title.crew"].directors == dict_df["name.basics"].nconst, how="inner")
    .select("primaryName").alias("Director")
)

display(highest_rated_comedy_movie_director_df)

DataFrame[primaryName: string]

**14**

For this last question we suspect that it was based on the fields provided in the _title.principals_ dataset (that we couldn't download at the beginning of this notebook since it causes OOM in Databricks...)

But we could easily imagine that the query to obtain the alternate titles would be something like:  
<br>
```
highest_rated_comedy_movie_titles_df = (
    highest_rated_comedy_movie
    .join(dict_df["title.principals"], on="tconst", how="inner")
    .select("primaryTitle", "alternatesTitle")
)

display(highest_rated_comedy_movie_titles_df)
```

---

## **Stream Processing**